<a href="https://colab.research.google.com/github/sheldonkemper/bank_of_england/blob/main/notebooks/processed/am_data_extraction_JPM_gpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Libraries**

In [ ]:
!pip install -q PyPDF2 > /dev/null 2>&1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.2 MB/s eta 0:00:00


In [ ]:
!pip install -q openai==0.28 > /dev/null 2>&1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 1.6 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.61.1
    Uninstalling openai-1.61.1:
      Successfully uninstalled openai-1.61.1


In [ ]:
!pip install -q python-dotenv > /dev/null 2>&1

In [ ]:
import re
import PyPDF2
import openai
import json
import pandas as pd
import time
import os
from dotenv import load_dotenv
from google.colab import userdata

In [ ]:
load_dotenv()
openai_api_key = os.getenv("Openai_key")

# **1. Data extraction**

In [ ]:
def extract_sections_from_pdf(pdf_path):
    """
    Reads a PDF file and extracts text from all pages.
    Then segregates the text into two sections based on markers:
    - 'Management Discussion' section (md_section)
    - 'Question and Answer' section (QNA_section)

    Parameters:
        pdf_path (str): The file path to the PDF.

    Returns:
        md_section (str): Extracted text for the Management Discussion section.
        QNA_section (str): Extracted text for the Question and Answer section.
    """
    # Open the PDF file in binary mode
    with open(pdf_path, 'rb') as pdf_file:
        # Create a PDF reader object
        pdf_reader = PyPDF2.PdfReader(pdf_file)
        num_pages = len(pdf_reader.pages)

        # Extract text from all pages
        full_text = ""
        for page_num in range(num_pages):
            page = pdf_reader.pages[page_num]
            page_text = page.extract_text()
            full_text += page_text + "\n"

    # Optionally, clean up extra whitespace/newlines
    full_text = re.sub(r'\n+', '\n', full_text)

    # Convert text to lowercase for marker searching (you can retain original text for extraction)
    text_lower = full_text.lower()

    # Define markers for splitting sections
    # Adjust these markers if your PDF uses slightly different headings
    md_marker = "management discussion"
    qna_marker = "question and answer"

    # Find the starting index of each section in the text
    md_start = text_lower.find(md_marker)
    qna_start = text_lower.find(qna_marker)

    if md_start == -1:
        raise ValueError("Management Discussion section marker not found in the PDF.")
    if qna_start == -1:
        raise ValueError("Question and Answer section marker not found in the PDF.")

    # Extract the sections based on the identified markers
    # We assume that the Management Discussion section comes first
    md_section = full_text[md_start:qna_start].strip()
    QNA_section = full_text[qna_start:].strip()

    return md_section, QNA_section

In [ ]:
if __name__ == "__main__":
    # Specify the path to your PDF file
    pdf_path = "/content/4q24-earnings-transcript.pdf"

    # Extract the sections
    md_section, QNA_section = extract_sections_from_pdf(pdf_path)

    # Optionally, convert the extracted text into datasets (e.g., as a list of lines)
    md_dataset = md_section.split("\n")
    qna_dataset = QNA_section.split("\n")

FileNotFoundError: [Errno 2] No such file or directory: '/content/4q24-earnings-transcript.pdf'

In [ ]:
    # Display a sample from each dataset
    print("=== Management Discussion Section Sample ===")
    for line in md_dataset[:5]:
        print(line)

In [ ]:
    print("\n=== Question and Answer Section Sample ===")
    for line in qna_dataset[:5]:
        print(line)

In [ ]:
# Convert md_dataset to DataFrame before saving
md_df = pd.DataFrame(md_dataset, columns=['Text'])
md_df.to_csv("MD_1Q23.csv", index=False)

In [ ]:
qna_dataset

In [ ]:
from google.colab import userdata


In [ ]:
openai.api_key = userdata.get('Openai_key')

In [ ]:
def convert_qna_dataset_to_text(qna_dataset):
    """
    Converts a list of Q&A pair strings into a single text string.

    Parameters:
        qna_dataset (list): List of strings, where each string represents a Q&A pair.

    Returns:
        str: A single string containing all Q&A pairs separated by newlines.
    """
    # Join the list elements using a newline separator
    text = "\n".join(qna_dataset)
    return text

# Example usage:
if __name__ == "__main__":
    # Convert the qna_dataset to a single text string
    qna_text = convert_qna_dataset_to_text(qna_dataset)
    print("Converted Q&A Text:")
    print(qna_text)

In [ ]:
def extract_operator_segments(qna_dataset):
    """
    Processes the qna_dataset to extract text between consecutive occurrences of the word Operator.

    Steps:
      1. Check if the word Operator exists in the text.
      2. Find all occurrences of Operator.
      3. Extract text between each consecutive pair of Operator occurrences.
      4. Create a DataFrame with two columns:
         - 'Question_Number': a count starting at 1.
         - 'Text': the text between consecutive occurrences.

    Parameters:
        qna_dataset (str): The input text containing multiple occurrences of Operator.

    Returns:
        pd.DataFrame: A DataFrame with the extracted segments.
    """
    # Step 1: Check if 'Operator' exists in the dataset.
    if "Operator" not in qna_dataset:
        print("The word 'Operator' is not found in the dataset.")
        return pd.DataFrame(columns=["Question_Number", "Text"])

    # Step 2: Find all occurrences of 'Operator'
    matches = list(re.finditer(r"Operator", qna_dataset))

    # Check if there are at least two occurrences to form a segment.
    if len(matches) < 2:
        print("Not enough occurrences of 'Operator' to extract segments.")
        return pd.DataFrame(columns=["Question_Number", "Text"])

    segments = []
    # Step 3: Extract text between consecutive occurrences.
    for i in range(len(matches) - 1):
        # Get the end index of the current occurrence and start index of the next occurrence.
        start = matches[i].end()
        end = matches[i+1].start()
        segment_text = qna_dataset[start:end].strip()
        segments.append(segment_text)

    # Step 4: Create the DataFrame.
    df = pd.DataFrame({
        "Question_Number": list(range(1, len(segments) + 1)),
        "Text": segments
    })
    return df

In [ ]:
if __name__ == "__main__":
    result_df = extract_operator_segments(qna_text)
    print(result_df)

In [ ]:
result_df

In [ ]:
result_df.to_csv("QNA_output.csv", index=False)

In [ ]:
def extract_info(text):
    """
    This function sends a prompt to the GPT-4 Turbo model asking it to extract
    specific fields from the provided text. The model is expected to return a JSON
    with the following keys:
    - Name of the first person
    - Role of the first person
    - All text that the first person said
    - Name of the second person
    - Role of the second person
    - All text that the second person said
    """
    prompt = f"""
    The text is conversation between two people. Please Extract the following information from the text below:

    - Name of the first person
    - Role of the first person
    - All text that the first person said
    - Name of the second person
    - Role of the second person
    - All text that the second person said


    The output should have all text both the persons said in the text.

    Provide the response in JSON format with keys exactly as:
    "Name of the first person", "Role of the first person", "All text that the first person said", "Name of the second person", "Role of the second person", "All text that the second person said".

    Text: {text}
    """
    try:
        response = openai.ChatCompletion.create(
            model="gpt-4-turbo",
            messages=[
                {"role": "system", "content": "You are a helpful assistant that extracts structured information from text."},
                {"role": "user", "content": prompt}
            ],
            response_format={"type": "json_object"}, # Set output to JSON format
            max_tokens=4000,  # Adjust tokens based on your text size
            temperature=0  # Keep it deterministic
        )
        content = response['choices'][0]['message']['content']
        # Attempt to parse the JSON response
        result = json.loads(content)
    except Exception as e:
        print(f"Error processing text: {e}")
        # Return a dictionary with None values in case of error
        result = {
            "Name of the first person": None,
            "Role of the first person": None,
            "All text that the first person said": None,
            "Name of the second person": None,
            "Role of the second person": None,
            "All text that the second person said": None
        }
    return result

In [ ]:
# List to store processed results
processed_results = []

# Loop through each row in result_df
for idx, row in result_df.iterrows():
    text = row['Text']
    info = extract_info(text)
    processed_results.append(info)
    # Optional: sleep to respect rate limits (adjust the delay as needed)
    time.sleep(1)

# Convert the list of dictionaries to a DataFrame
processed_df = pd.DataFrame(processed_results)

# Display the processed DataFrame
processed_df.head()

In [ ]:
processed_df.info()

In [ ]:
processed_df.to_csv("QNA_4Q24.csv", index=False)